In [19]:
### 최대한 외부 라이브러리를 사용하지 않고, 코드를 작성했습니다.

import re
from math import log
import pandas as pd

class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

    def preprocessing(self, sequences):

        result = []  # 전처리한 데이터를 담을 리스트 선언.
        for sequence in sequences :   # 입력된 각 문장에 아래와 같은 전처리 실행.
            sequence = sequence.lower()  # 소문자화.
            sequence = re.sub('[^a-zA-Zㄱ-ㅎ가-힣]', ' ', sequence)  # 특수문자 제거
            sequence = sequence.split()   # 문장을 공백 기준으로 리스트로 변환.
            result.append(sequence)   # result 리스트에 추가.
        return result
  
    def fit(self, sequences):
        self.fit_checker = False

        sentences = self.preprocessing(sequences)  # 클래스 내부 함수로 전처리함.
        num = 1   # vocab 구성에서 self.word_dict을 사용. 숫자0은 oov로 할당되어 숫자 1부터 key값 할당.
        for sentence in sentences :
            for word in sentence:
                if word not in self.word_dict :  # self.word_dict의 딕셔너리에 해당 어휘가 없을 시, 추가.
                    self.word_dict[word] = num
                    num +=1

        self.fit_checker = True
  

    def transform(self, sequences):
        result = []
        preprocessed_sentences = self.preprocessing(sequences)   # 클래스 내부 함수 호출하여 전처리함.

        if self.fit_checker:   # 클래스 내부 fit함수로 vocab이 미리 구성될 경우.
            for sentence in preprocessed_sentences :
                sub_result = []  
                for word in sentence :
                    if word in self.word_dict :   
                        sub_result.append(self.word_dict[word])   # 입력 문장이 어휘집합에 있을 경우 해당 토큰을 부여.
                    else:
                        sub_result.append(self.word_dict['oov'])   # 입력 문장이 어휘집합에 없을 경우 "OOV"를 부여.
                result.append(sub_result)
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
      
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result



class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

        self.N = 0   # 입력 문장의 수
        self.vocab = []   # 입력 시퀀스로 토큰 집합 구성
        self.fit_checker = False

        self.idf_list = []  # idf 값을 저장할 리스트
        self.tf_list = []   # tf 값을 저장할 리스트
        self.tfidf_matrix = []  # tf-idf 값 저장할 리스트
        
    def make_vocab(self, tokenized) :  
        # Tokenizer 클래스로 전처리된 시퀀스로 토큰 집합 구성

        self.N = len(tokenized)  # 입력 시퀀스의 수 선언
        vocab_list = []   
        for i in range(self.N) :  # 입력 시퀀스를 하나의 리스트로 받음.
            vocab_list += tokenized[i]
            
        vocab = list(set(vocab_list))  # 중복값 제거
        vocab.sort()    # 토큰 집합 정렬
        self.vocab = vocab   # 클래스 내에 토큰 집합 저장


    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        # Tokenizer 클래스를 사용하여 입력 데이터를 전처리함.

        self.make_vocab(tokenized)  # 토큰 집합 구성
        

        result = []  # idf 값을 저장할 리스트 선언.

        for j in range(len(self.vocab)) :
            token = self.vocab[j]   # 토큰 집합을 하나씩 호출
            df = 0
            for k in tokenized :
                df += token in k    # 해당 토큰이 등장한 문장(시퀀스)의 수
            result.append(log(self.N/(df+1)))  # idf 값 구함.

        self.idf_list = result    # idf 값 저장.
        self.fit_checker = True


    def transform(self, sequences):

        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            
            tf_result = []   # tf 값을 저장할 리스트.
            tfidf_result = []  # tfidf 값을 저장할 리스트.

            for i in range(self.N) :
                tf_result.append([])  # nested list 형태로 출력하기 위해 리스트 선언.
                tfidf_result.append([])  # nested list 형태로 출력하기 위해 리스트 선언.

                doc = tokenized[i]  # 시퀀스를 순서대로 호출.
                for j in range(len(self.vocab)) :  # 토큰 집합을 순서대로 호출.
                    token = self.vocab[j]

                    tf = doc.count(token)   # 각 시퀀스에 해당 토큰의 수를 셈.
                    tf_result[-1].append(tf)  # 

                    idf = self.idf_list[token - 1]  
                    # 토큰 '1'에 대한 idf_list의 값은 0번째이므로 1씩 빼줘서 인덱싱함.
                    tfidf_result[-1].append(tf*idf)
                    

            self.tf_list = tf_result   # tf 값 저장
            self.tfidf_matrix = tfidf_result   # TF-IDF 값 저장.
            
            return self.tfidf_matrix
 
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

  
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)   



### testing

# tokenizer = Tokenizer()
# tokenizer.fit_transform(['I go to school.', 'I LIKE pizza!'])
# tokenizer.transform(['I go to home.', 'do you like chicken?',])

# tokenizer = Tokenizer()
# tfidf = TfidfVectorizer(tokenizer)
# test = ['I go to home.', 'do you like chicken?', 'i go to korea', 'chicken chicken i like', ' korea korea i you']
# tfidf.fit_transform(test)